# File Transfer via Jump Server

This notebook provides a user interface for transferring files through a jump server using SSH tunneling. It supports both sending and receiving files with various transfer methods.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import os
from jump_server_transfer import NamedPipeSSHBridge, TransferMethod, FileTransferFactory
import threading
import time
from functools import partial

In [ ]:
# Style definitions
style = HTML("""
<style>
.widget-label { min-width: 150px !important; }
.widget-text, .widget-password, .widget-select { width: 300px !important; }
</style>
""")
display(style)

In [ ]:
# Create widgets with default values
hostname = widgets.Text(
    value='45.145.74.109',
    description='Jump Server:',
    placeholder='Enter hostname or IP',
    style={'description_width': 'initial'}
)

port = widgets.IntText(
    value=5222,
    description='SSH Port:',
    style={'description_width': 'initial'}
)

username = widgets.Text(
    value='root',
    description='Username:',
    placeholder='Enter SSH username',
    style={'description_width': 'initial'}
)

password = widgets.Password(
    description='Password:',
    placeholder='Enter SSH password',
    style={'description_width': 'initial'}
)

local_path = widgets.Text(
    description='Local Path:',
    placeholder='Enter file/folder path',
    style={'description_width': 'initial'}
)

remote_target = widgets.Text(
    value='file_transfer_default',
    description='Remote Target:',
    placeholder='Enter remote identifier',
    style={'description_width': 'initial'}
)

operation = widgets.RadioButtons(
    options=['send', 'receive'],
    value='receive',
    description='Operation:',
    disabled=False,
    style={'description_width': 'initial'}
)

buffer_size = widgets.IntText(
    value=8192,
    description='Buffer Size:',
    style={'description_width': 'initial'}
)

transfer_method = widgets.Dropdown(
    options=[(method.value, method) for method in TransferMethod],
    value=TransferMethod.SSH_NAMED_PIPE,
    description='Transfer Method:',
    style={'description_width': 'initial'}
)

status = widgets.Output()

progress = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    orientation='horizontal',
    style={'description_width': 'initial'}
)

submit = widgets.Button(
    description='Start Transfer',
    button_style='primary',
    style={'description_width': 'initial'}
)

In [ ]:
def validate_inputs():
    """Validate user inputs before starting transfer"""
    errors = []
    
    if not hostname.value:
        errors.append("Jump server hostname is required")
    if not username.value:
        errors.append("SSH username is required")
    if not password.value:
        errors.append("SSH password is required")
    if not local_path.value:
        errors.append("Local path is required")
    if operation.value == 'send' and not os.path.exists(local_path.value):
        errors.append(f"Local file does not exist: {local_path.value}")
    if port.value <= 0 or port.value > 65535:
        errors.append("Invalid port number (must be between 1 and 65535)")
    if buffer_size.value <= 0:
        errors.append("Buffer size must be greater than 0")
        
    return errors

def update_progress(progress: widgets.FloatProgress, value):
    """Update progress bar with smooth animation"""
    # Ensure value is between 0 and 100
    value = max(0, min(100, value))
    # Update the progress bar widget
    progress.value = value
    # Update bar style based on progress
    if value == 100:
        progress.bar_style = 'success'
    elif value == 0:
        progress.bar_style = 'info'
    else:
        progress.bar_style = 'info'

update_progress_partial = partial(update_progress, progress=progress)

def on_submit(b):
    """Handle submit button click"""
    # Clear previous status
    status.clear_output()
    
    # Disable submit button and reset progress
    submit.disabled = True
    progress.value = 0
    progress.bar_style = 'info'
    
    with status:
        try:
            # Validate inputs
            errors = validate_inputs()
            if errors:
                print("\n".join([f"Error: {err}" for err in errors]))
                return
            
            # Create bridge instance
            bridge = FileTransferFactory.create_bridge(
                transfer_method.value,
                hostname.value,
                port.value,
                username.value,
                password.value
            )
            
            print(f"Connecting to {hostname.value}...")
            if not bridge.connect():
                print("Connection failed!")
                return
                
            try:
                # Configure bridge
                if hasattr(bridge, 'PIPE_NAME'):
                    bridge.PIPE_NAME = f"transfer_pipe_{remote_target.value}"
                
                # Start transfer
                print(f"\nStarting {operation.value} operation...")
                print(f"Transfer method: {transfer_method.value.value}")
                print(f"Buffer size: {buffer_size.value} bytes")
                
                if operation.value == 'send':
                    success = bridge.send_file(local_path.value, buffer_size.value, update_progress_partial)
                else:  # receive
                    success = bridge.receive_file(local_path.value, buffer_size.value, update_progress_partial)
                
                if success:
                    print("\nTransfer completed successfully!")
                    progress.bar_style = 'success'
                else:
                    print("\nTransfer failed!")
                    progress.bar_style = 'danger'
                    
            finally:
                bridge.close()
                print("Connection closed")
                
        except Exception as e:
            print(f"\nError: {str(e)}")
            progress.bar_style = 'danger'
        finally:
            submit.disabled = False

submit.on_click(on_submit)

In [ ]:
# Layout the interface
connection_box = widgets.VBox([
    widgets.HBox([hostname, port]),
    widgets.HBox([username, password])
])

transfer_box = widgets.VBox([
    widgets.HBox([local_path, remote_target]),
    widgets.HBox([transfer_method, buffer_size]),
    operation
])

display(widgets.VBox([
    widgets.HTML("<h3>Connection Settings</h3>"),
    connection_box,
    widgets.HTML("<h3>Transfer Settings</h3>"),
    transfer_box,
    widgets.HBox([submit, progress]),
    status
]))